In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


### Lets take a look at the post questions data

In [4]:
# Construct a reference to the "posts_answers" table
q_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
q_table = client.get_table(q_table_ref)

# Preview the first five lines of the "posts_answers" table
questions = client.list_rows(q_table, max_results=400).to_dataframe()
questions

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,320268,Html.ActionLink doesn’t render # properly,<p>When using Html.ActionLink passing a string...,NaN,0,0,NaT,2008-11-26 10:42:37.477000+00:00,0.0,2009-02-06 20:13:54.370000+00:00,NaT,None,NaN,Paulo,NaN,None,1,0,asp.net-mvc,390
1,324003,Primitive recursion,<p>how will i define the function 'simplify' ...,NaN,0,0,NaT,2008-11-27 15:12:37.497000+00:00,0.0,2012-09-25 19:54:40.597000+00:00,2012-09-25 19:54:40.597000+00:00,Marcin,1288.0,None,41000.0,None,1,0,haskell|lambda|functional-programming|lambda-c...,497
2,390605,While vs. Do While,<p>I've seen both the blocks of code in use se...,390608.0,0,0,NaT,2008-12-24 01:49:54.230000+00:00,2.0,2008-12-24 03:08:55.897000+00:00,NaT,None,NaN,Unkwntech,115.0,None,1,0,language-agnostic|loops,11262
3,413246,Protect ASP.NET Source code,<p>Im currently doing some research in how to ...,NaN,0,0,NaT,2009-01-05 14:23:51.040000+00:00,0.0,2009-03-24 21:30:22.370000+00:00,2009-01-05 14:42:28.257000+00:00,Tom Anderson,13502.0,Velnias,NaN,None,1,0,asp.net|deployment|obfuscation,4823
4,454921,"Difference between ""int[] myArray"" and ""int my...",<blockquote>\n <p><strong>Possible Duplicate:...,454928.0,0,0,NaT,2009-01-18 10:22:52.177000+00:00,0.0,2009-01-18 10:30:50.930000+00:00,2017-05-23 11:49:26.567000+00:00,None,-1.0,Evan Fosmark,49701.0,None,1,0,java|arrays,798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1222707,Java Math Functions for BigDecimal,<blockquote>\n <p><strong>Possible Duplicate:...,NaN,0,3,NaT,2009-08-03 14:36:29.330000+00:00,NaN,2011-12-14 21:25:24.257000+00:00,2017-05-23 11:47:47.183000+00:00,None,-1.0,None,57627.0,None,1,2,java|math|bigdecimal,1890
396,1223212,How to weight in full-text search,"<p>I want to search on a phrase, for example ""...",NaN,0,1,NaT,2009-08-03 16:10:53.250000+00:00,NaN,2011-10-21 19:05:56.113000+00:00,2011-10-21 19:05:56.113000+00:00,None,905093.0,None,149834.0,None,1,2,full-text-search|containstable,761
397,1223655,Set Project Location - Already Contains a Project,<p>I have a .NET 1.1 solution I'm opening in V...,NaN,0,2,NaT,2009-08-03 17:45:54.343000+00:00,NaN,2009-08-03 17:45:54.343000+00:00,NaT,None,NaN,None,11112.0,None,1,2,visual-studio-2003|iis-5,1294
398,1229866,File.lastModified() returns stale date on Goog...,"<p>So <a href=""https://stackoverflow.com/quest...",NaN,0,4,NaT,2009-08-04 20:51:46.173000+00:00,NaN,2009-08-04 20:51:46.173000+00:00,2017-05-23 12:30:43.553000+00:00,None,-1.0,None,93995.0,None,1,2,java|http|google-app-engine|servlets,542


In [5]:
questions.isnull().sum()

id                            0
title                         0
body                          0
accepted_answer_id          396
answer_count                  0
comment_count                 0
community_owned_date        399
creation_date                 0
favorite_count              282
last_activity_date            0
last_edit_date               88
last_editor_display_name    348
last_editor_user_id          99
owner_display_name          291
owner_user_id                60
parent_id                   400
post_type_id                  0
score                         0
tags                          0
view_count                    0
dtype: int64

In [6]:
questions = questions.drop(["accepted_answer_id","community_owned_date","parent_id"],axis=1)

In [7]:
import pandas as pd
questions['tags'] = questions['tags'].str.split('|')
questions = questions.explode('tags')
questions

,id,title,body,answer_count,comment_count,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count
0,320268,Html.ActionLink doesn’t render # properly,<p>When using Html.ActionLink passing a string...,0,0,2008-11-26 10:42:37.477000+00:00,0.0,2009-02-06 20:13:54.370000+00:00,NaT,None,NaN,Paulo,NaN,1,0,asp.net-mvc,390
1,324003,Primitive recursion,<p>how will i define the function 'simplify' ...,0,0,2008-11-27 15:12:37.497000+00:00,0.0,2012-09-25 19:54:40.597000+00:00,2012-09-25 19:54:40.597000+00:00,Marcin,1288.0,None,41000.0,1,0,haskell,497
1,324003,Primitive recursion,<p>how will i define the function 'simplify' ...,0,0,2008-11-27 15:12:37.497000+00:00,0.0,2012-09-25 19:54:40.597000+00:00,2012-09-25 19:54:40.597000+00:00,Marcin,1288.0,None,41000.0,1,0,lambda,497
1,324003,Primitive recursion,<p>how will i define the function 'simplify' ...,0,0,2008-11-27 15:12:37.497000+00:00,0.0,2012-09-25 19:54:40.597000+00:00,2012-09-25 19:54:40.597000+00:00,Marcin,1288.0,None,41000.0,1,0,functional-programming,497
1,324003,Primitive recursion,<p>how will i define the function 'simplify' ...,0,0,2008-11-27 15:12:37.497000+00:00,0.0,2012-09-25 19:54:40.597000+00:00,2012-09-25 19:54:40.597000+00:00,Marcin,1288.0,None,41000.0,1,0,lambda-calculus,497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,1229866,File.lastModified() returns stale date on Goog...,"<p>So <a href=""https://stackoverflow.com/quest...",0,4,2009-08-04 20:51:46.173000+00:00,NaN,2009-08-04 20:51:46.173000+00:00,2017-05-23 12:30:43.553000+00:00,None,-1.0,None,93995.0,1,2,google-app-engine,542
398,1229866,File.lastModified() returns stale date on Goog...,"<p>So <a href=""https://stackoverflow.com/quest...",0,4,2009-08-04 20:51:46.173000+00:00,NaN,2009-08-04 20:51:46.173000+00:00,2017-05-23 12:30:43.553000+00:00,None,-1.0,None,93995.0,1,2,servlets,542
399,1231519,How to change schema name in Crystal report 11?,<p>The report is being designed in Crystal Rep...,0,0,2009-08-05 06:31:18.817000+00:00,NaN,2015-04-15 17:20:06.610000+00:00,2015-04-15 17:20:06.610000+00:00,None,305.0,vijeta,NaN,1,2,oracle,5676
399,1231519,How to change schema name in Crystal report 11?,<p>The report is being designed in Crystal Rep...,0,0,2009-08-05 06:31:18.817000+00:00,NaN,2015-04-15 17:20:06.610000+00:00,2015-04-15 17:20:06.610000+00:00,None,305.0,vijeta,NaN,1,2,crystal-reports,5676


In [8]:
tag_counts = questions['tags'].value_counts()
tag_counts.head(60)

tags
c#                     43
.net                   35
asp.net                27
iphone                 22
java                   16
wpf                    15
javascript             15
visual-studio-2008     10
sharepoint             10
windows                 9
xml                     9
visual-studio           9
vb.net                  9
jquery                  8
apache-flex             8
objective-c             7
python                  7
c++                     7
cocoa-touch             6
subsonic                6
sql-server-2005         6
web-services            6
sql-server              6
xaml                    6
php                     6
asp.net-mvc             6
windows-vista           5
winforms                5
html                    5
silverlight             5
internet-explorer       5
subsonic3               5
flash                   4
remoting                4
wcf                     4
security                4
debugging               4
windows-mobile          4
arrays 

In [18]:
filtered_df = questions[questions['tags'] == 'database']
filtered_df

,id,title,body,answer_count,comment_count,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count
37,789167,"Visual Studio 2008, Local Database, cannot mod...",<p>I just installed Visual Studio 2008 and SQL...,0,3,2009-04-25 15:40:05.020000+00:00,NaN,2017-01-09 17:21:58.300000+00:00,2017-01-09 17:21:58.300000+00:00,None,1599619.0,None,45991.0,1,1,database,1106
105,1018558,Rails find :include in a multi DB environment,<p>I am making a site in RoR and I am inside a...,0,3,2009-06-19 15:21:37.697000+00:00,NaN,2018-06-28 16:19:09.990000+00:00,2018-06-28 16:19:09.990000+00:00,None,50198.0,None,125806.0,1,1,database,566
349,927190,Problem using two xa-datasources with Jbpm,"<p>I'm using Seam, JBoss Jbpm and Resteasy. I ...",0,2,2009-05-29 17:28:28.077000+00:00,NaN,2012-08-16 09:21:31.383000+00:00,2009-05-29 17:49:48.507000+00:00,None,114311.0,None,114311.0,1,2,database,2007


In [19]:
filtered_df['body']

37     <p>I just installed Visual Studio 2008 and SQL...
105    <p>I am making a site in RoR and I am inside a...
349    <p>I'm using Seam, JBoss Jbpm and Resteasy. I ...
Name: body, dtype: object

In [20]:
import pandas as pd

# Assuming you have a DataFrame named 'filtered_df' with 'id' and 'body' columns
# Iterate over each row in the DataFrame and print 'id' and 'body' columns
for index, row in filtered_df.iterrows():
    print(f"ID: {row['id']}")
    print(row['body'])
    print()  # Add an empty line to separate entries


ID: 789167
<p>I just installed Visual Studio 2008 and SQL Server 2008 on my machine - in that order.  </p>

<p>I note that VS insisted on installing SQL Server 2005 Express in order to be able to create a local database in by web projects.  </p>

<p>In any case, whenever I add a local database and go to create a diagram, I can't seem to modify the <code>Custom</code> table view.  </p>

<p>For example, I like to see the column name, data type, nullable and default value columns.</p>

<p>Whenever I click <code>"Table View" - "Modify Custom"</code>, nothing happens!  </p>

<p>Strangely, this action works fine in SQL Server 2008.  </p>

<p>Has anyone else ever encountered this - and (hopefully) found a solution?</p>

ID: 1018558
<p>I am making a site in RoR and I am inside a multi DB environment. By that, I mean that some of my models are linked to MSSQL tables, and some others are linked to MYSQL tables.</p>

<p>It works well for the most part, but when I use the 'include' option in a fin